In [1]:
import socket
import threading
import logging

clients = []
usernames = {}  
logging.basicConfig(filename='server_log.txt', level=logging.INFO)


def broadcast(message, sender_socket, username):
    for client in clients:
        if client != sender_socket:
            try:
                client.send(f"{username}: {message}".encode())
            except:
                clients.remove(client)


def handle_client(client_socket, username):
    while True:
        try:
            message = client_socket.recv(1024)
            if message:
                decoded_message = message.decode()
                logging.info(f"{username}: {decoded_message}")
                print(f"Received from {username}: {decoded_message}")
                broadcast(message, client_socket, username)
            else:
                clients.remove(client_socket)
                del usernames[client_socket]
        except:
            clients.remove(client_socket)
            del usernames[client_socket]
            break

def start_server():
    server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server.bind(('localhost', 1234))
    server.listen()

    print("Server is listening for connections...")

    while True:
        client_socket, client_addr = server.accept()
        clients.append(client_socket)

        username = client_socket.recv(1024).decode()
        print(f"{username} connected")
        logging.info(f"{username} connected")

        welcome_message = f"Welcome to the chat, {username}!"
        client_socket.send(welcome_message.encode())

        broadcast(f"{username} joined the chat.", client_socket, "Server")

        usernames[client_socket] = username

        client_thread = threading.Thread(target=handle_client, args=(client_socket, username))
        client_thread.start()

if __name__ == "__main__":
    start_server()  


Server is listening for connections...
daryna connected
Received from daryna: Hello
